In [27]:
!pip install pandas
!pip install numpy
!pip install matplotlib


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 19.5 MB/s  0:00:00m0:00:0100:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 15.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 30.6 MB/s  0:00:00
Using cached pyparsing-3.2.5-py3-none-any.whl (113 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [matplotlib]7 [matplotlib]


In [4]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]


In [2]:
import pandas as pd

oct = pd.read_excel('october.xlsx', sheet_name=None)
nov = pd.read_excel('november.xlsx', sheet_name=None)
dec = pd.read_excel('december.xlsx', sheet_name=None)

In [17]:
from datetime import datetime, time

floor1 = ['Lobby/Atrium', 'Competition Pool', 'Instructional Pool', 'Spa',
          'Classroom 105', 'Spectator Seating', 'Classroom 120', 'MAC',
          'Golf Simulator', 'Climbing Wall', 'OA']

floor2 = ['2nd Floor Fitness', '2nd Floor Cardio Pool Side',
          '2nd Floor Cardio MAC Side', '2nd Floor Soft Seating',
          'MPR Mind Body', 'MPR Spin', 'MPR Functional Training',
          'MPR Group Fitness']

floor3 = ['3rd Floor Wood Courts', '3rd Floor Tennis Courts',
          '3rd Floor Fitness Area - Wood Side',
          '3rd Floor Fitness Area - Tennis Side',
          '3rd Floor Fitness Platforms']

floor4 = ['4th Floor FTA (Turf & Racks)', '4th Floor Fitness Queenax',
          '4th Floor Cardio MODs', 'Racquetball & Squash Courts', 'Track']


def parse_sheet_date(sheet_name):
    sheet_name = sheet_name.strip()
    # delete this sheet
    if sheet_name == "Template": # Special case
        return datetime(2025, 10, 1)
    if sheet_name == "1012025":  # Special case
        return datetime(2025, 10, 1)
    if sheet_name == "1112025":  # Special case
        return datetime(2025, 11, 1)
    if sheet_name == "1212025":  # Special case
        return datetime(2025, 12, 1)
    if len(sheet_name) == 3:  # e.g. "102" → October 2
        month = int(sheet_name[0]) + 9  # 10, 11, 12
        day = int(sheet_name[1:])
    elif len(sheet_name) == 4:  # e.g. "1209" → Dec 9
        month = int(sheet_name[:2])
        day = int(sheet_name[2:])
    else:
        raise ValueError(f"Cannot parse sheet name: {sheet_name}")
    return datetime(2025, month, day)


def clean_month(path):
    raw = pd.read_excel(path, sheet_name=None)
    cleaned_list = []

    for sheet_name, df in raw.items():
        date = parse_sheet_date(sheet_name)
        weekday = date.strftime("%A")
        if sheet_name == "Template":
            continue

        time_map = {'Unnamed: 0': 'Area',
                    'Unnamed: 1': '6.5',
                    'Unnamed: 2': '7.5',
                    'Unnamed: 3': '8.5',
                    'Unnamed: 4': '9.5',
                    'Unnamed: 5': '10.5',
                    'Unnamed: 6': '11.5',
                    'Unnamed: 7': '12.5',
                    'Unnamed: 8': '13.5',
                    'Unnamed: 9': '14.5',
                    'Unnamed: 10': '15.5',
                    'Unnamed: 11': '16.5',
                    'Unnamed: 12': '17.5',
                    'Unnamed: 13': '18.5',
                    'Unnamed: 14': '19.5',
                    'Unnamed: 15': '20.5',
                    'Unnamed: 16': '21.5',
                    'Unnamed: 17': '22.5',
                    'Unnamed: 18': '23.5',
                    'Unnamed: 19': 'Totals'
    }

        df = df.rename(columns=time_map)

        # Then first column is AREA
        df = df.rename(columns={df.columns[0]: 'AREA'})

        # Convert time columns to numeric
        time_cols = df.columns[1:]  # everything except AREA
        df[time_cols] = df[time_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

        # Compute floor totals
        floors = pd.DataFrame(columns=time_cols, index=["Floor 1", "Floor 2", "Floor 3", "Floor 4"])

        floors.loc["Floor 1"] = df[df["AREA"].isin(floor1)][time_cols].sum()
        floors.loc["Floor 2"] = df[df["AREA"].isin(floor2)][time_cols].sum()
        floors.loc["Floor 3"] = df[df["AREA"].isin(floor3)][time_cols].sum()
        floors.loc["Floor 4"] = df[df["AREA"].isin(floor4)][time_cols].sum()

        floors = floors.reset_index().rename(columns={"index": "floor"})

        # Melt into long form
        long = floors.melt(id_vars=["floor"], var_name="time", value_name="count")
        long["date"] = date
        long["weekday"] = weekday

        cleaned_list.append(long)

    return pd.concat(cleaned_list, ignore_index=True)

# ---------------------------------------------------------
# MAIN PIPELINE
# ---------------------------------------------------------
oct_df = clean_month("october.xlsx")
nov_df = clean_month("november.xlsx")
dec_df = clean_month("december.xlsx")

full = pd.concat([oct_df, nov_df, dec_df], ignore_index=True)

weekly_avg = (
    full.groupby(["weekday", "floor", "time"])["count"]
        .mean()
        .reset_index()
)

weekday_order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
weekly_avg["weekday"] = pd.Categorical(weekly_avg["weekday"], categories=weekday_order, ordered=True)
weekly_avg = weekly_avg.sort_values(["weekday", "floor", "time"])

# ---------------------------------------------------------
# Print sample output
# ---------------------------------------------------------
print("FULL LONG DATA (first 10 rows):")
print(full.head(10), "\n")

print("WEEKLY AVERAGES (first 10 rows):")
print(weekly_avg.head(10))


FULL LONG DATA (first 10 rows):
     floor time count       date    weekday
0  Floor 1  6.5   4.0 2025-10-01  Wednesday
1  Floor 2  6.5  25.0 2025-10-01  Wednesday
2  Floor 3  6.5  33.0 2025-10-01  Wednesday
3  Floor 4  6.5  22.0 2025-10-01  Wednesday
4  Floor 1  7.5   1.0 2025-10-01  Wednesday
5  Floor 2  7.5  40.0 2025-10-01  Wednesday
6  Floor 3  7.5  60.0 2025-10-01  Wednesday
7  Floor 4  7.5  36.0 2025-10-01  Wednesday
8  Floor 1  8.5   6.0 2025-10-01  Wednesday
9  Floor 2  8.5  52.0 2025-10-01  Wednesday 

WEEKLY AVERAGES (first 10 rows):
   weekday    floor  time      count
76  Monday  Floor 1  10.5   4.846154
77  Monday  Floor 1  11.5   5.846154
78  Monday  Floor 1  12.5   6.230769
79  Monday  Floor 1  13.5   7.230769
80  Monday  Floor 1  14.5   8.769231
81  Monday  Floor 1  15.5   8.307692
82  Monday  Floor 1  16.5  12.538462
83  Monday  Floor 1  17.5  12.923077
84  Monday  Floor 1  18.5  16.307692
85  Monday  Floor 1  19.5   8.615385
